This notebook allows you to run a word alignment assessment and view all the results in their entirety, to help with debugging.

In [1]:
from pathlib import Path
from typing import Optional, Literal

import modal
from pydantic import BaseModel
import pandas as pd

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_rows', 500)
# from app import Assessment

In [2]:
class Assessment(BaseModel):
    assessment: Optional[int] = None
    revision: int
    reference: int
    type: Literal["word-alignment"]

In [9]:
! pip install ipywidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.8/137.8 kB 3.1 MB/s eta 0:00:003.4 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 3.4 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.7/149.7 kB 3.2 MB/s eta 0:00:00 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.4/117.4 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 784.3/784.3 kB 3.3 MB/s eta 0:00:003.6 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.3/384.3 kB 2.9 MB/s eta 0:00:00 MB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.3/94.3 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 3.4 MB/s eta 0:00:00m eta 0:00:010:00:01
  Using cached nest_asyncio-1.5.6-py3-none-any.whl (5.2 kB)
  Using cached matplotlib_inline-0.1.6-py3-none-any.whl (9.4 kB)
  Using cached tornado-6.2-cp37-abi3-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_6

In [29]:
stub = modal.Stub(
    name="run-word-alignment-debug",
    image=modal.Image.debian_slim().pip_install(
        "pandas==1.4.3",
        "machine==0.0.1",
        "sil-machine[thot]>=0.8.3",
        "asyncio"
    ).copy(
        mount=modal.Mount.from_local_dir(
            local_path=Path("../../fixtures/"), remote_path=Path("/root/fixtures")
        ),
    ),
)

In [30]:
stub.run_word_alignment = modal.Function.from_name("word-alignment-test", "assess")

In [31]:
@stub.function(
            timeout=3600, 
            mounts=[
    *modal.create_package_mounts(["app", "word_alignment_steps"]),
    modal.Mount.from_local_dir(local_path="./", remote_path="/"),
])
def get_results(config):
    from app import Assessment
    assessment_config = Assessment(
            revision=config['revision'], 
            reference=config['reference'], 
            type='word-alignment'
            )
    results = modal.container_app.run_word_alignment.call(assessment_config, return_all_results = True)
    return results

In [32]:
def assess_draft(revision, reference):
    with stub.run():

        config = {
                'revision':revision, 
                'reference':reference, 
        }
        
        results = get_results.call(config=config)
    return results

In [53]:
revision = 11 # Isenye
reference = 163 # Hebrew Lemma

In [54]:
results = assess_draft(revision, reference)

/home/mark/.local/share/virtualenvs/aqua-api-qPfiOqBF/lib/python3.10/site-packages/rich/live.py:229: UserWarning: 
install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')
⠋ Initializing...

✓ Initialized. View app at https://modal.com/apps/ap-EfYEx8WIi5MbncFQglg3YD

✓ Created objects.
├── 🔨 Created mount ../../fixtures
├── 🔨 Created get_results.
├── 🔨 Created mount /home/mark/SIL/aqua-api/assessments/word_alignment/app.py
├── 🔨 Created mount /home/mark/SIL/aqua-api/assessments/word_alignment/word_alignment_steps
├── 🔨 Created mount /home/mark/SIL/aqua-api/assessments/word_alignment/word_alignment_steps
└── 🔨 Created mount .

/pkg/app.py:34: DeprecationError: 2023-02-08: The Mount constructor is deprecated. Use static factory method Mount.from_local_dir or Mount.from_local_file
⠋ Running (1/1 containers active)...

  mount=modal.Mount(
⠋ Running (1/1 containers active)...

...

2023-02-14T20:18:37+0000 Could not find app function run_translation_scores
⠋ Running (1/1 containers active)...

2023-02-14T20:18:37+0000 Could not find app function run_match_scores
⠋ Running (1/1 containers active)...

2023-02-14T20:18:37+0000 Could not find app function run_embedding_scores
⠋ Running (1/1 containers active)...

2023-02-14T20:18:37+0000 Could not find app function assess
⠋ Running (1/1 containers active)...

✓ App completed.

In [35]:
(results['embedding_score'] > 0).mean()

0.5975139703538767

In [56]:
results.shape

(9269073, 12)

In [55]:
results[:100]

,vref,source,target,alignment_score,alignment_count,co-occurrence_count,translation_score,avg_aligned,match_score,count,embedding_score,total_score
0,GEN 1:1,בְּ,hapo,0.714355,6.0,70.0,0.030334,0.085693,0.011913,122.0,0.295166,0.227492
1,GEN 1:1,בְּ,mwanzo,0.000000,0.0,12.0,0.000000,0.000000,0.002260,23.0,0.061096,0.012671
2,GEN 1:1,בְּ,mungu,0.000000,0.0,762.0,0.004360,0.000000,0.104945,1197.0,-0.195679,-0.017275
3,GEN 1:1,בְּ,aliumba,0.000000,0.0,1.0,0.000000,0.000000,0.000295,3.0,0.210938,0.042247
4,GEN 1:1,בְּ,mbingu,0.000000,0.0,36.0,0.000000,0.000000,0.007612,78.0,0.236694,0.048861
5,GEN 1:1,בְּ,na,0.000000,0.0,5806.0,0.043915,0.000000,0.375848,6483.0,0.129883,0.109929
6,GEN 1:1,בְּ,dunia,0.000000,0.0,83.0,0.000021,0.000000,0.014679,152.0,0.207397,0.04442
7,GEN 1:1,בְּ,.,0.000000,0.0,6773.0,0.043945,0.000000,0.444085,9050.0,0.467529,0.191112
8,GEN 1:1,רֵאשִׁ֖ית,hapo,0.433350,1.0,1.0,0.136353,1.000000,0.007874,2.0,0.067017,0.328919
9,GEN 1:1,רֵאשִׁ֖ית,mwanzo,0.000000,0.0,1.0,0.011177,0.000000,0.043956,4.0,0.077026,0.026432


In [49]:
total_scores_df = results[['vref', 'source', 'target', 'total_score']]
top_source_scores_df = total_scores_df.fillna(0)
top_source_scores_df = top_source_scores_df.loc[top_source_scores_df.groupby(['vref', 'source'], sort=False)['total_score'].idxmax(), :].reset_index(drop=True)

In [50]:
top_source_scores_df

,vref,source,target,total_score
0,GEN 1:1,בְּ,semuka,0.287766
1,GEN 1:1,רֵאשִׁ֖ית,semuka,0.369233
2,GEN 1:1,בָּרָ֣א,semuka,0.275195
3,GEN 1:1,אֱלֹהִ֑ים,eriibha,0.422554
4,GEN 1:1,אֵ֥ת,ndiiteemire,0.366293
...,...,...,...,...
31119,JON 4:11,לִ,.,0.207845
31120,JON 4:11,שְׂמֹאל֔,koworigheri,0.236335
31121,JON 4:11,וּ,",",0.293875
31122,JON 4:11,בְהֵמָ֖ה,cheetughu,0.061081


In [51]:
verse_scores_df = top_source_scores_df.groupby('vref', as_index=False, sort=False).mean()
verse_scores_df = verse_scores_df.fillna(0)

In [52]:
verse_scores_df['total_score'].mean()

0.3169891800050549